## **Assignment 8**

Develop a model for 20 news groups dataset from scikit-learn. Select 20% of data for test set.

Develop metric learning model with siamese network [3 points] and triplet loss [3 points] (from seminar). Use KNN and LSH (any library for approximate nearest neighbor search) for final prediction after the network was trained. [2 points]

! Remember, that LSH gives you a set of neighbor candidates, for which you have to calculate distances to choose top-k nearest neighbors.

Your quality metric = accuracy score [2 points if acc > 0.8 ]

In [0]:
import torch as tt
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
import pickle

from tqdm import tqdm_notebook 
import os

import re
from collections import defaultdict, Counter
from string import punctuation
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import pandas as pd
import numpy as np
import random

punct = punctuation+'«»—…“”*№–'

In [2]:
tt.cuda.is_available()

True

## Данные 

In [0]:
label = ['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

Так как у меня возникли проблемы с тем, чтобы импортировать и по-нормальному разбить данные из sklearn (простите, я что-то дико упли, наверно, да, я знаю...).
В любом случае, спасибо за наводку, я нашёл те же самые данные в Интернете. Это словари, где key - индекс газеты из списка выше, а значения - индексы текстов в списке all_texts, где хранятся все-все тексты. 

In [4]:
from google.colab import files
uploaded = files.upload()
# all_texts.pickle
# Его надо будет распоковать, не смог его загруить его в реп  полностью, пришлось зиппнуть

Saving all_texts.pickle to all_texts.pickle


In [5]:
from google.colab import files
uploaded = files.upload()
# x_train.pickle

Saving x_test.pickle to x_test.pickle


In [6]:
from google.colab import files
uploaded = files.upload()
# x_test.pickle

Saving x_train.pickle to x_train.pickle


In [7]:
from google.colab import files
uploaded = files.upload()
# x_valid.pickle

Saving x_valid.pickle to x_valid.pickle


In [0]:
with open('./all_texts.pickle', 'rb') as f:
    all_texts = pickle.load(f)
    
with open('./x_test.pickle', 'rb') as f:
    x_test = pickle.load(f)
    
with open('./x_train.pickle', 'rb') as f:
    x_train = pickle.load(f)
    
with open('./x_valid.pickle', 'rb') as f:
    x_valid = pickle.load(f)


In [9]:
all_texts[0]

'From: mathew <mathew@mantis.co.uk> Subject: Alt.Atheism FAQ: Atheist Resources Summary: Books, addresses, music -- anything related to atheism Keywords: FAQ, atheism, books, music, fiction, addresses, contacts Expires: Thu, 29 Apr 1993 11:57:19 GMT Distribution: world Organization: Mantis Consultants, Cambridge. UK. Supersedes: <19930301143317@mantis.co.uk> Lines: 290  Archive-name: atheism/resources Alt-atheism-archive-name: resources Last-modified: 11 December 1992 Version: 1.0                                Atheist Resources                        Addresses of Atheist Organizations                                       USA  FREEDOM FROM RELIGION FOUNDATION  Darwin fish bumper stickers and assorted other atheist paraphernalia are available from the Freedom From Religion Foundation in the US.  Write to:  FFRF, P.O. Box 750, Madison, WI 53701. Telephone: (608) 256-8900  EVOLUTION DESIGNS  Evolution Designs sell the "Darwin fish".  It\'s a fish symbol, like the ones Christians stick on

Создаем пары pos, neg, anc

In [0]:
def TestSet(data):
    x = []
    y = []
    for cls in data:
        for txt in data[cls]:
            x.append(txt)
            y.append(cls)
    return x, y


def CustomDataset(data, all_texts):
    
    all_size = [len(data[i]) for i in data]
    
    indices_anc = np.random.choice(list(range(len(label))), size=sum(all_size))
    
    pair = []
    y = []

    for anc_ind in indices_anc:
    
        neg_ind = np.random.choice([l for l in range(len(label)) if anc_ind != l])
    
        pos, anc = [np.random.choice(data[anc_ind]) for _ in range(2)]
        neg = np.random.choice(data[neg_ind])
    
        pair.append((anc, pos, neg))
        y.append(anc_ind)
    
    return pair, y


train_x, train_y = CustomDataset(x_train, all_texts)
valid_x, valid_y = CustomDataset(x_valid, all_texts)
test_x, test_y = TestSet(x_test)

In [11]:
train_x[0], train_y[0]

((827, 696, 4557), 1)

Build vocab

In [0]:
def tokenizer(text):
    words = [word.strip(punct) for word in text.lower().split()]
    return words

In [0]:
vocab = Counter()
max_len = 0

for text in all_texts:
    tok = tokenizer(text)
    vocab.update(tok)
    if len(tok) > max_len: max_len = len(tok)


In [14]:
max_len

11821

In [0]:
vocab = [i for i in vocab if vocab[i] > 10]

In [0]:
vocab += ['<pad>', '<eos>']

In [17]:
len(vocab)

22415

In [18]:
vocab.index('text')

1648

In [0]:
def tokenizer2(text, vocab):
    
    tensor = tt.zeros(11821).long()
    tensor = tensor.fill_(22413)  # padding
    index = 0

    
    for word in text.lower().split():

        try:
            tensor[index] = vocab.index(word.strip(punct))
            index += 1
        except: continue
       
    tensor[index] = 22414

    return tensor, index

In [20]:
new_text = []
max_len = 0

for text in tqdm_notebook(all_texts):
    tensor, index = tokenizer2(text, vocab)
    new_text.append(tensor) 
    if index > max_len: max_len = index


In [21]:
max_len

11246

In [0]:
new_text = [text[:max_len] for text in new_text]

In [23]:
new_text[0]

tensor([    0,     1,     2,  ..., 22413, 22413, 22413])

In [0]:
del all_texts

# Model

In [0]:
def _train_epoch(model, iterator, optimizer, curr_epoch):

    model.train()

    running_loss = 0

    n_batches = len(iterator)
    iterator = tqdm_notebook(iterator, total=n_batches, desc='epoch %d' % (curr_epoch), leave=True)

    for i, batch in enumerate(iterator):
        optimizer.zero_grad()

        loss = model(batch).sum()
        loss.backward()
        optimizer.step()

        curr_loss = loss.data.cpu().detach().item()
        
        loss_smoothing = i / (i+1)
        running_loss = loss_smoothing * running_loss + (1 - loss_smoothing) * curr_loss

        iterator.set_postfix(loss='%.5f' % running_loss)

    return running_loss

def _test_epoch(model, iterator):
    model.eval()
    epoch_loss = 0

    n_batches = len(iterator)
    with tt.no_grad():
        for batch in iterator:
            loss = model(batch).sum()
            epoch_loss += loss.data.item()

    return epoch_loss / n_batches


def nn_train(model, train_iterator, valid_iterator, optimizer, n_epochs=100,
          scheduler=None, early_stopping=0):

    prev_loss = 100500
    es_epochs = 0
    best_epoch = None
    history = pd.DataFrame()

    for epoch in range(n_epochs):
        train_loss = _train_epoch(model, train_iterator, optimizer, epoch)
        valid_loss = _test_epoch(model, valid_iterator)

        valid_loss = valid_loss
        print('validation loss %.5f' % valid_loss)

        record = {'epoch': epoch, 'train_loss': train_loss, 'valid_loss': valid_loss}
        history = history.append(record, ignore_index=True)

        if early_stopping > 0:
            if valid_loss > prev_loss:
                es_epochs += 1
            else:
                es_epochs = 0

            if es_epochs >= early_stopping:
                best_epoch = history[history.valid_loss == history.valid_loss.min()].iloc[0]
                print('Early stopping! best epoch: %d val %.5f' % (best_epoch['epoch'], best_epoch['valid_loss']))
                break

            prev_loss = min(prev_loss, valid_loss)

In [0]:
def triplet_loss(anchor_embed, pos_embed, neg_embed):
    return F.cosine_similarity(anchor_embed, neg_embed) - F.cosine_similarity(anchor_embed, pos_embed)


class MyModel(nn.Module):
    
    def __init__(self, vocab_size, embed_size):
        super(MyModel, self).__init__()
        
        self.embedding = nn.Embedding(vocab_size, embed_size)
        tt.nn.init.uniform_(self.embedding.weight)
        self.fc = nn.Linear(embed_size, 1)
        
    def branch(self, x):
        x = self.embedding(x)
        x = self.fc(x)
        return x
        
    def forward(self, batch):
        
        anchor = new_text[batch[0]]
        pos = new_text[batch[1]]
        neg =  new_text[batch[2]]
        
        anchor = self.branch(anchor)
        pos = self.branch(pos)
        neg = self.branch(neg)
        
        x = triplet_loss(anchor, pos, neg)
        
        return x


In [27]:
batch_size = 32
embed_size = 100

model = MyModel(len(vocab), embed_size)

optimizer = optim.Adam(model.parameters())
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=5)

nn_train(model, train_x, valid_x, optimizer, n_epochs=3, scheduler=scheduler, early_stopping=2)



validation loss -0.00000



validation loss 0.00000



validation loss 0.00000
Early stopping! best epoch: 0 val -0.00000


# KNN

Собираем данные для кнн.

In [29]:
!pip install annoy

     |████████████████████████████████| 645kB 3.4MB/s 
  Created wheel for annoy: filename=annoy-1.16.3-cp36-cp36m-linux_x86_64.whl size=297346 sha256=767a7e8af96a27705aa2f1bf76dd816d57da99cbe87ede854c5d06a21d4e8dc6
  Stored in directory: /root/.cache/pip/wheels/f3/01/54/6ef760fe9f9fc6ba8c19cebbe6358212b5f3b5b0195c0b813f
Successfully built annoy


In [0]:
import annoy


def knn(train_x, ntree, vec_len):

    vector_length = vec_len
    t = annoy.AnnoyIndex(vector_length)

    for i, v in zip(range(len(train_x)), train_x):
        emb = model.branch(new_text[v[0]]).squeeze(1)
        t.add_item(i, emb)
        
    t.build(ntree)
    return t

In [0]:
vec_len = len(model.branch(new_text[0]).squeeze(1))

In [32]:
t = knn(train_x, 10, vec_len)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.
  import sys


In [0]:
cos = nn.CosineSimilarity(dim=0)

In [34]:
tt.tensor(t.get_item_vector(11))

tensor([-6.7261, -6.9284, -7.4556,  ..., -7.8568, -7.8568, -7.8568])

Так как в формуле loss мы использовали косинусную близость, здесь ее тоже нужно использовать. так как annoy не даёт возможности выбрать коссинусную близость, то мы возьмем какое-то количество соседей от annoy и уже посчитаем расстояния до них по нашей метрике.

In [0]:
def predict(test_x, model, n, t):

    pred = []

    for item in tqdm_notebook(test_x):
        
        item = model.branch(new_text[item]).squeeze(1)
    
        res = {}
        for i in t.get_nns_by_vector(item, n):
            res[i] = int(cos(tt.tensor(t.get_item_vector(i)), item))
    
        ind = sorted(res.items(), key=lambda kv: kv[1])
        a = Counter([train_y[i[0]] for i in ind])
        pred.append(max(a, key=a.get))
    
    return pred

In [36]:
pred = predict(test_x, model, 5, t)

In [39]:
acc = accuracy_score(test_y, pred)

acc

0.05947955390334572

Я не знаю, зачем я это пишу, но все же уточню, что я понимаю, что результат плохой, что надо было как минимум использовать элмо, а как максимум разобраться, но у меня не было просто времени сделать хорошо. Або это есть мой потолок!
(вотэва)